# Importing the Data and The Libraries

In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from tensorflow.keras import regularizers
from tensorflow.keras import initializers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import decomposition
from sklearn.model_selection import StratifiedKFold



from sklearn.decomposition import KernelPCA, PCA

df = pd.read_csv("HumidityDataset.csv")

C:\Users\sherwin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (1,2,4,5,6,7,8,9,10,11,12,13,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Dropping all the libraires and selecting only rows that start after the 2010

In [2]:
df.drop(["longitude","latitude", "WaveHeight", "WavePeriod", "MeanWaveDirection", "Hmax","QC_Flag"],inplace = True, axis = 1)
df = df.iloc[331371:]
buoy_ident = { 'M2':1 , 'M3': 2, 'M4':3, 'M5': 4, 'M6': 5}
# df.station_id.nunique()

In [3]:
# Selecting only the values of rows that have buoys ident as M2 to M6
df = df.loc[df.station_id.isin(buoy_ident.keys()) ]
df = df.drop(["time"], axis = 1)
# df.station_id.unique()

In [4]:
df = df.replace({ 'station_id': buoy_ident})
df = df.dropna(axis = 1, how='all')
df.reset_index(inplace = True)
df.dropna(inplace = True, how = 'all')
# df.head()

In [5]:
df = df.dropna()
df.drop('index', axis = 1, inplace = True)
# df.reset_index(inplace= True)
# df.isna()
# drop('index',axis = 1, inplace = True)

In [6]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(df.iloc[:,:-1].values,df.iloc[:,-1].values, test_size=0.2)
print(X_train1.shape)

class CustomScaler():
    
        def __init__(self, X_Scale, y_Scale ) :
        
            self.X_Scale =   X_Scale 
            self.y_Scale =   y_Scale
            
            self.x_num =     [np.mean(self.X_Scale[:,i]) for i in range(self.X_Scale.shape[1])]
            self.x_maxs =    [np.max(self.X_Scale[:,i]) for i in range(self.X_Scale.shape[1])]
            self.x_mins =    [np.min(self.X_Scale[:,i]) for i in range(self.X_Scale.shape[1])]
            
            self.y_num =     np.mean(self.y_Scale)
            self.y_max_min = np.max(self.y_Scale) - np.min(self.y_Scale)

        def scaleX(self, x_value):
            x = x_value.copy()
            for i in range(x.shape[1]):
                
                x[:,i]= (x[:,i] - self.x_num[i])/(self.x_maxs[i]-self.x_mins[i])
            return x

        def inverseScaleX(self, x_value):
            x = x_value.copy()
            
            for i in range(x.shape[1]):
                x[:,i]= (x[:,i] * (self.x_maxs[i]-self.x_mins[i])) + self.x_num[i]
            return x



        

        def scaleY(self, y_value):
            y = y_value.copy()
            ys = (y - self.y_num)/(self.y_max_min)
            return ys

        def inverseScaleY(self, y_value):
            y = y_value.copy()
            ys = (y * self.y_max_min) + self.y_num
            return ys


        



custom_scaler = CustomScaler(X_train1,y_train1)
    
X_train = custom_scaler.scaleX(X_train1.copy())
y_train = custom_scaler.scaleY(y_train1.copy())
X_train = np.asarray(X_train).astype('float32')
y_train = np.asarray(y_train).astype('float32')
transform = None


print(X_train[-70000: -1])

print(y_train1.shape)


(68427, 8)
[[ 2.5003654e-01  2.6198825e-02  3.6794931e-01 ... -1.8317389e-01
  -5.2942269e-02 -1.4113867e-01]
 [ 2.5003654e-01  4.3781243e-02  1.4572708e-01 ...  8.3871566e-02
   1.2963034e-01  1.5350418e-01]
 [ 3.6535286e-05  3.1327028e-02 -4.9316180e-01 ... -2.1158297e-01
  -1.5252733e-01 -3.5542440e-01]
 ...
 [-4.9996346e-01  9.1400288e-02  3.9572707e-01 ...  1.4637156e-01
   1.1303284e-01  1.8028989e-01]
 [ 2.5003654e-01  9.2865489e-02  6.8381876e-03 ... -1.3557082e-03
   1.2548096e-01 -1.5899582e-01]
 [-2.4996346e-01  4.4513844e-02 -5.2093959e-01 ... -1.3203752e-01
  -1.6497546e-01 -9.6495822e-02]]
(68427,)


In [7]:
# # transformer = KernelPCA(0.95, kernel='linear').fit(X_train)
# # transformer = PCA(0.99).fit(X_train)
# transformer = decomposition.TruncatedSVD(n_components=7, algorithm='arpack').fit(X_train)

# X_train = transformer.transform(X_train)
# print(X_train)

In [8]:

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [10]:
def build_model():    
    model = keras.Sequential()
    model.add(Dense(10, activation = "relu",input_dim = X_train.shape[1], name = "layer1"))
    model.add(Dense(20, activation = "relu", name = "layer2"))
    model.add(Dense(20, activation = "relu", name = "layer5"))
    model.add(Dense(20, activation = "relu", name = "layer6"))

    model.add(Dense(1, kernel_initializer='normal', name = "layer7"))
    model.compile(loss="mean_squared_error", optimizer='SGD')
    return model

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

# regression_model = KerasRegressor(build_fn=build_model, epochs=150, batch_size=1024, verbose=1,callbacks=[es])
# history=estimator.fit(X_train, y_train)
from sklearn.model_selection import cross_val_score
regressor = KerasRegressor(build_fn = build_model, batch_size = 1024, epochs = 10000)
accuracies = cross_val_score(estimator = regressor, X = X_train, y = y_train,scoring='r2',cv = 5, n_jobs = 1)
mean = accuracies.mean()
variance = accuracies.std()

# kfold = StratifiedKFold(n_splits=5, shuffle=True)
# cvscores = []
# for train, test in kfold.split(X_train, y_train):
#     history=estimator.fit(np.asarray(X_train[train]).astype('float32'),np.asarray(y_train[train]).astype('float32'))
#     scores = estimator.evaluate(np.asarray(X_train[test]).astype('float32'),np.asarray(y_train[test]).astype('float32'), verbose=0)
#     print("%s: %.2f%%" % (estimator.metrics_names[1], scores[1]*100))
#     cvscores.append(scores[1] * 100)
# print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))

Epoch 1/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0333
Epoch 2/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0333
Epoch 3/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0332
Epoch 4/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0332
Epoch 5/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0331
Epoch 6/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0331
Epoch 7/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0330
Epoch 8/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0330
Epoch 9/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0329
Epoch 10/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0328
Epoch 11/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0328
Epoch 12/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0327
Epoch 13/100
54/54 [=================

54/54 [==============================] - 0s 3ms/step - loss: 0.0328
Epoch 3/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0327
Epoch 4/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0326
Epoch 5/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0326
Epoch 6/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0325
Epoch 7/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0324
Epoch 8/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0324
Epoch 9/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0323
Epoch 10/100
54/54 [==============================] - 0s 4ms/step - loss: 0.0322
Epoch 11/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0322
Epoch 12/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0321
Epoch 13/100
54/54 [==============================] - 0s 4ms/step - loss: 0.0320
Epoch 14/100
54/54 [============================

54/54 [==============================] - 0s 3ms/step - loss: 0.0327
Epoch 4/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0327
Epoch 5/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0326
Epoch 6/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0326
Epoch 7/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0326
Epoch 8/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0326
Epoch 9/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0326
Epoch 10/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0326
Epoch 11/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0325
Epoch 12/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0325
Epoch 13/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0325
Epoch 14/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0325
Epoch 15/100
54/54 [===========================

54/54 [==============================] - 0s 3ms/step - loss: 0.0331
Epoch 5/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0330
Epoch 6/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0330
Epoch 7/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0329
Epoch 8/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0329
Epoch 9/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0329
Epoch 10/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0328
Epoch 11/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0328
Epoch 12/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0327
Epoch 13/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0327
Epoch 14/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0327
Epoch 15/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0326
Epoch 16/100
54/54 [==========================

54/54 [==============================] - 0s 3ms/step - loss: 0.0327
Epoch 6/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0327
Epoch 7/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0326
Epoch 8/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0326
Epoch 9/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0326
Epoch 10/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0325
Epoch 11/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0325
Epoch 12/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0324
Epoch 13/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0324
Epoch 14/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0324
Epoch 15/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0323
Epoch 16/100
54/54 [==============================] - 0s 3ms/step - loss: 0.0323
Epoch 17/100
54/54 [=========================

In [12]:
variance 

0.13773677118087843

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])

plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
if transform:
    y_pred = estimator.predict(transformer.transform(custom_scaler.scaleX(np.asarray(X_test1).astype('float32'))))
else :
    y_pred = estimator.predict((custom_scaler.scaleX(np.asarray(X_test1).astype('float32'))))

# y_pred = max_min*y_pred + y_min
print(mean_squared_error(custom_scaler.inverseScaleY(y_pred), y_test1))
y_test1

In [ ]:
np.column_stack((y_test1,custom_scaler.inverseScaleY(y_pred)))

In [ ]:
predTest = scaler.inverse_transform(np.column_stack((X_test, y_pred)))
ogTest = scaler.inverse_transform(np.column_stack((X_test, y_test)))
np.column_stack((predTest[:, -1], ogTest[:, -1]))